## Start by setting week number
#### Wait for ESPN data update until at least Wednesday morning, but run latest by Thursday night

https://www.espn.com/nfl/stats/team/_/stat/total/season/2021/seasontype/2

In [1]:
SET_WEEK = 10
week_name = 'week_' + str(SET_WEEK)
print(week_name)

week_10


In [2]:
from scrapers.scrape_2021 import get_2021_data
import pandas as pd
import numpy as np
import pickle

import warnings
warnings.filterwarnings('ignore')

### Models

In [3]:
rf = '../modeling/rf.pkl'
gbm = '../modeling/gbm.pkl'
lgb = '../modeling/lgb.pkl'
catboost = '../modeling/catboost.pkl'

with open(rf, 'rb') as file:
    RF = pickle.load(file)
    
with open(gbm, 'rb') as file:
    GBM = pickle.load(file)
    
with open(lgb, 'rb') as file:
    LGB = pickle.load(file)
    
with open(catboost, 'rb') as file:
    CATBOOST= pickle.load(file)

### 2021 data

In [4]:
%%time

X = get_2021_data()

# from boruta elimination method:
keep_cols = ['offense_points_per_game', 'offense_passing_AVG', 'offense_passing_YDS/G', 'offense_passing_RTG', 
             'offense_receiving_AVG', 'offense_rushing_YDS/G', 'defense_points_per_game', 'defense_passing_AVG', 
             'defense_receiving_AVG', 'defense_rushing_YDS/G', 'defense_passing_SYL_per_game', 
             'offense_downs_Third Downs_ATT_per_game', 'defense_downs_Third Downs_ATT_per_game', 
             'defense_downs_Fourth Downs_ATT_per_game', 'defense_pass_to_rush_ratio', 
             'defense_downs_First Downs_penalty_ratio', 'offense_passing_TD_per_game', 'offense_pass_TD_to_INT']

X = X[['Team', 'games_played', 'season'] + keep_cols].copy()

X.head(2)

Wall time: 17.7 s


,Team,games_played,season,offense_points_per_game,offense_passing_AVG,offense_passing_YDS/G,offense_passing_RTG,offense_receiving_AVG,offense_rushing_YDS/G,defense_points_per_game,...,defense_receiving_AVG,defense_rushing_YDS/G,defense_passing_SYL_per_game,offense_downs_Third Downs_ATT_per_game,defense_downs_Third Downs_ATT_per_game,defense_downs_Fourth Downs_ATT_per_game,defense_pass_to_rush_ratio,defense_downs_First Downs_penalty_ratio,offense_passing_TD_per_game,offense_pass_TD_to_INT
0,Dallas Cowboys,9,2021,31.6,8.3,294.3,109.1,11.9,139.6,21.7,...,12.0,101.2,17.333333,12.888889,12.111111,1.444444,1.523810,0.116279,2.444444,3.666667
1,Baltimore Ravens,9,2021,25.7,7.8,260.0,91.7,12.2,154.1,24.1,...,13.1,88.2,15.222222,12.666667,11.888889,1.111111,1.732984,0.079096,1.555556,1.750000


In [5]:
X.fillna(0, inplace = True)
X.replace([np.inf, -np.inf], 0, inplace=True)

In [6]:
X['games_played'].unique().tolist()

[9, 10]

In [7]:
# X[X['games_played'] == 8]

In [8]:
predictions = X[['Team', 'season']].copy()
predictions['week_marker'] = week_name

predictions['pred_rf'] = RF.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_gbm'] = GBM.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_lgb'] = LGB.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_catboost'] = CATBOOST.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_mean'] = (predictions['pred_rf'] + predictions['pred_gbm'] + predictions['pred_lgb'] + predictions['pred_catboost']) / 4

predictions.sort_values('pred_mean', ascending = False, inplace = True)
predictions.head(3)

,Team,season,week_marker,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean
0,Dallas Cowboys,2021,week_10,0.273961,0.257705,0.361182,0.296343,0.297298
4,Buffalo Bills,2021,week_10,0.269474,0.229194,0.301972,0.311969,0.278152
2,Tampa Bay Buccaneers,2021,week_10,0.250621,0.204785,0.321695,0.292671,0.267443


In [9]:
predictions.to_csv('weekly_runs/' + week_name + '.csv', index = False)

### Check weekly run

In [10]:
predictions

,Team,season,week_marker,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean
0,Dallas Cowboys,2021,week_10,0.273961,0.257705,0.361182,0.296343,0.297298
4,Buffalo Bills,2021,week_10,0.269474,0.229194,0.301972,0.311969,0.278152
2,Tampa Bay Buccaneers,2021,week_10,0.250621,0.204785,0.321695,0.292671,0.267443
8,Arizona Cardinals,2021,week_10,0.256082,0.219344,0.268080,0.269775,0.253320
5,Los Angeles Rams,2021,week_10,0.240087,0.155579,0.264129,0.274907,0.233676
7,Las Vegas Raiders,2021,week_10,0.158140,0.103531,0.175531,0.219172,0.164094
19,Green Bay Packers,2021,week_10,0.170975,0.085578,0.056715,0.234318,0.136896
12,San Francisco 49ers,2021,week_10,0.108478,0.070423,0.144782,0.200925,0.131152
14,New England Patriots,2021,week_10,0.124386,0.085113,0.082694,0.180440,0.118158
13,Cincinnati Bengals,2021,week_10,0.101636,0.057303,0.096016,0.197207,0.113041
